**Поместите данный файл в директорию, где находится библиотека CityGeoTools для корректного вызова метода DataImputer**

In [ ]:
import gevent.monkey
gevent.monkey.patch_all()

import osm2geojson
import geopandas as gpd
import pandas as pd
import numpy as np
import grequests
import requests
import shapely
import sys

# import warnings
# warnings.filterwarnings('ignore')

from tqdm import tqdm

In [ ]:
# геокодирование - получаем координаты x,y по адресу объекта

with open("./keys.txt") as f:
    api_keys = f.read().splitlines()

urls = []
api_key_counter = 0
for i, adr in enumerate(list(buildings_gkh["address"])):
    if i%24500 == 0 and i != 0:
        api_key_counter += 1
    urls.append(yandex_create_urls(adr, api_keys[api_key_counter]))
yandex_geocoded_data = yandex_geocode(urls)
gkh_geometry = pd.DataFrame(yandex_geocoded_data)

In [ ]:
gkh_geometry = pd.read_json("/var/essdata/IDU/other/mm_22/dpo/gkh_geometry.json")

In [ ]:
# создаем объект shapely (Point) из координат x,y
# конвертируем DataFrame в GeoDataFrame, присваем проекцию

gkh_geometry = gkh_geometry[(gkh_geometry["x"] !=0) & (gkh_geometry["y"] !=0)]
buildings_gkh["geometry"] = pd.DataFrame(gkh_geometry).apply(
    lambda row: shapely.geometry.Point(row.y, row.x), axis=1
    )

# удаляем из датасета объекты, для которых в ходе геокодирования не подобрались координаты
buildings_gkh = gpd.GeoDataFrame(buildings_gkh).set_crs(4326).to_crs(32636)
buildings_gkh = buildings_gkh.dropna(subset=["geometry"]).reset_index(drop=True)

# присоединяем полигональную геометрию к датасету с Реформы ЖКХ
buildings_gkh["geometry"] = buildings_gkh["geometry"].buffer(10)
buildings_gkh = buildings_gkh.sjoin(buildings[["geometry"]], how="inner")
buildings_gkh = buildings_gkh.reset_index().drop_duplicates(subset=["index"]).drop(["index"], axis=1)

In [ ]:
buildings_gkh["heating_type"].unique()
# Центральное - 1
# Не заполнено - 2
# Автономная - 3

# One-Hot encoding

In [ ]:
# Прелдобработка датасета для восстановления данных:
# 1. Признаки должны быть информативны.
# 2. Все значения должны быть числовые.
# 3. Набор данных должен быть сохранен в geojson файл.


# Удаляем малоинформативный признаки с индексами и адресом.
buildings_gkh = buildings_gkh.drop(["address", "index_right"], axis=1)

# Приводим значения признаков area_total, area_residential и heating_type к числовым
def replace_comma(value):
    if type(value) == str:
        return float(value.replace(",", "."))
    else:
        return value
        
buildings_gkh["area_total"] = buildings_gkh["area_total"].apply(lambda x: replace_comma(x))
buildings_gkh["area_residential"] = buildings_gkh["area_residential"].apply(lambda x: replace_comma(x))
buildings_gkh["heating_type"] = buildings_gkh["heating_type"].apply(lambda x: 0 if x == "Отсутствует" else 1)

# Сохраняет GeoDataFrame в geojson
# buildings_gkh.to_file("./buildings_gkh.geojson", driver="GeoJSON")

In [ ]:
buildings_gkh = gpd.read_file("./buildings_gkh.geojson")

In [ ]:
buildings_gkh

In [ ]:
# path to folder to CityGeoTools
folder = "/var/essdata/IDU/other/mm_22/teaching_2023/CityGeoTools"
sys.path.append(folder)

from data_imputer.imputer import DataImputer
dataset_path = "./buildings_gkh.geojson"
imputer = DataImputer(dataset_path)
imputer.add_neighbors_features()
full_data = imputer.multiple_imputation(save_models=False, save_logs=False)

In [ ]:
full_data

In [ ]:
full_data[buildings_gkh.columns].isna().sum()

In [ ]:
import matplotlib.pyplot as plt
full_data[buildings_gkh.columns].drop(['geometry'], axis=1).hist(bins=50,figsize=(15,15))
plt.show()